# Blockchain Main Classes

In [3]:
import hashlib
import pickle
import copy
import timeit
import matplotlib.pyplot as plt

## Completar la clase RSA_KEY

In [4]:
class rsa_key:            
    def __init__(self,bits_modulo=2048,e=2**16+1):
        def generatePQ(NdeBits):
            r1=randint(0,2^(NdeBits/2));
            r2=randint(0,2^(NdeBits/2));
            p=next_probable_prime(r1);
            while gcd(e,p-1)!=1:
                print( ".");
                p=next_probable_prime(p+2);

            q=next_probable_prime(r2);
            while gcd(e,q-1)!=1:
                print( "+");
                q=next_probable_prime(q+2);
            return p, q

        
        self.primeP, self.primeQ = generatePQ(bits_modulo)
        self.publicExponent = e
        self.privateExponent = mod(e^-1, (self.primeP-1)*(self.primeQ-1))
        self.modulus = self.primeP * self.primeQ
        self.privateExponentModulusPhiP = mod(self.privateExponent, self.primeP-1)
        self.privateExponentModulusPhiQ = mod(self.privateExponent, self.primeQ-1)
        self.inverseQModulusP = mod(self.primeQ^-1, self.primeP)
        
    def sign(self,message):
        cp=mod(message,self.primeP); #Necesario para que trabaje en Zp, si no tarda mucho
        cq=mod(message,self.primeQ); #Necesario para que trabaje en Zq, si no tarda mucho
        mp=Integer(mod(cp ^ self.privateExponentModulusPhiP,self.primeP));
        mq=Integer(mod(cq ^ self.privateExponentModulusPhiQ,self.primeQ));
        h=mod((mp-mq)*Integer(self.inverseQModulusP),self.modulus);
        m_tcr=mod((mq+self.primeQ*h),self.modulus);
        return m_tcr
    
    def sign_slow(self,message):
        return pow(message,self.privateExponent,self.modulus)

## Completar la clase RSA_PUBLIC_KEY

In [5]:
class rsa_public_key:
    def __init__(self, modulus, public):
        self.publicExponent = public
        self.modulus = modulus
    def verify(self, message, signature):
        return message == pow(signature,self.publicExponent,self.modulus)

## Completar la clase TRANSACTION

In [6]:
class transaction:
    def __init__(self, message, RSAkey, signature):
        self.public_key = RSAkey
        self.message = message
        self.signature = signature
    def verify(self):
        return self.public_key.verify(self.message, self.signature)

## Completar la clase BLOCK

In [7]:
class block:
    def __init__(self):
        self.block_hash = 0
        self.previous_block_hash = 0
        #self.transaction = transaction(0, rsa_key())
        self.seed = -1
        self.rango = 2^32
        
    def defecto(self, actual, previous, transaction, seed):
        self.block_hash = actual
        self.previous_block_hash = previous
        self.transaction = transaction
        self.seed = seed
        
    def genesis(self,transaction):
        d = 8
        previous_block_hash = 0
        valido = False
        while (not valido) and self.seed < self.rango:
            self.seed += 1
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(self.seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            valido = h < (2^(256-d))
                    
        self.previous_block_hash = 0
        self.block_hash = h
        self.transaction = transaction
        
    def next_block(self, transaction):
        d = 8
        previous_block_hash = self.block_hash
        self.seed = -1
        valido = False
        while not valido and self.seed < self.rango:
            self.seed += 1
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(self.seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            valido = h < (2^(256-d))
        self.previous_block_hash = self.block_hash
        self.block_hash = h
        self.transaction = transaction
        
    def verify_block(self):
        d = 8
        condicion1 = self.transaction.verify()
        condicion2 = self.block_hash < (2^(256-d))
        condicion3 = self.previous_block_hash < (2^(256-d))
        
        entrada=str(self.previous_block_hash)
        entrada=entrada+str(self.transaction.public_key.publicExponent)
        entrada=entrada+str(self.transaction.public_key.modulus)
        entrada=entrada+str(self.transaction.message)
        entrada=entrada+str(self.transaction.signature)
        entrada=entrada+str(self.seed)
        h=int(hashlib.sha256(entrada.encode()).hexdigest(),16)
        
        condicion4 = self.block_hash == h
        
        if not condicion1:
            print("Transaccion no valida")
            
        if not condicion2:
            print("Hash de tamaño no correcto")
        
        if not condicion3:
            print("Hash del bloque anterior malo")
        
        if not condicion4:
            print("La seed esta mal")
        
        return condicion1 and condicion2 and condicion3 and condicion4

## Completar la clase BLOCK_CHAIN

In [8]:
class block_chain:
    def __init__(self,transaction):
        first_block = block()
        first_block.genesis(transaction)
        self.list_of_blocks = [first_block]
        
    def add_block(self,transaction):
        ultimo = copy.deepcopy(self.list_of_blocks[-1])
        ultimo.next_block(transaction)
        self.list_of_blocks.append(ultimo)
        
    def verify(self):
        d = 8
        bien = True
        index = 0
        
        bien = self.list_of_blocks[0].previous_block_hash == 0
        if not bien:
            print(f'Hay un fallo en la iteración {index}: bloque inicial con hash diferente de 0')
            return bien
        
        while index < len(self.list_of_blocks) and bien:
            actual = self.list_of_blocks[index]
            bien = actual.verify_block()
            index += 1
        
        if not bien:
            print(f'Hay un fallo en la iteración {index}: causa bloque no valido.')
            return bien
        
        index = 0
        while index < len(self.list_of_blocks) - 1 and bien:
            index += 1
            bien = self.list_of_blocks[index-1].block_hash == self.list_of_blocks[index].previous_block_hash
        
        if not bien:
            print(f'Hay un fallo en la iteración {index}: causa mal hash conectado.')
            return bien
        
        index = -1
        while index < len(self.list_of_blocks) - 1 and bien:
            index += 1
            
            previous_block_hash = self.list_of_blocks[index].previous_block_hash
            transaction = self.list_of_blocks[index].transaction
            seed = self.list_of_blocks[index].seed
            
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            
            bien = h == self.list_of_blocks[index].block_hash
        
        if not bien:
            print(f'Hay un fallo en la iteración {index}: causa mala seed')
        
        return bien
    
    def falso(self, transaction):
        ultimo = copy.deepcopy(self.list_of_blocks[-1])
        ultimo.next_block(transaction)
        ultimo.block_hash = 0
        self.list_of_blocks.append(ultimo)

# Examen

In [9]:
modulus = 17609275718897705797693944927749287888098244670668123740639071754645310623129481163456157175152182448859534944406503550063680832132391217767945865892791446701045300663542231860126167946761334266310775875215974588540130332715652080668109605032193091094381386062991019989923988755261615483526208151537459902377091178704741629061653924653619269147925439410819658680918614713976644830989135022928100235342831933477257862730290709342557366103932579480407236923267397181905380652289162708336347529812558300142444957614597991783784594577910954543574671560060468339162220918942208119768314466039044657851139303014324171751473
expPublic = 65537
rsa = rsa_public_key(modulus, expPublic)

In [11]:
mensaje = 74978285425673526727744618033707927825626444397979562261062263846703991521232
seed = 6306714462597086943864166428997195756246047584818563356624388711605293265512004643270621363757311633853686407911299042681324831062031939006448252465864316154210553408929313124594310633253211597300372011199757340195192141062673992021263319153394339390091671489462185019891530862567107337292757654243736433524940103034897680086348329193893311876271379794330527836764028061347576294800408673070408006725853291638167052717629792975737036226866898699689308998507847894294564707798890152714902931899430409016696396691053922354964984394806772752312690374469466150646983385381906139685745256122316098042154355375822111997070
transaccion = transaction(mensaje, rsa, seed)
print('Transaccion correcta:',transaccion.verify())

Transaccion correcta: True


In [12]:
actual = 32913054361703563669153793576202057517067728388035989298541810241209692097
previo = 312242924808911849479051425103604997481886996854157223258323582102268035232
seed = 3141683475901211307547989473453913452684045358938510538598389160685116442738799022
bloque = block()
bloque.defecto(actual, previo, transaccion, seed)

In [13]:
bloque.verify_block()

La seed esta mal


False